In [2]:
import pandas as pd
import pysubgroup as ps



In [84]:
# Load the dataset
csv_file_path = './Data/tournaments.csv'
df = pd.read_csv(csv_file_path)


In [61]:
df.head()

,id_card,name_card,amount_card,price_card,energy_type_card,type_card,combo_type_id,combo_type_name,id_player,name_player,...,country_tournament,year_tournament,month_tournament,day_tournament,valid_rotation_at_tournament,rotation_name,year_begin,month_begin,day_begin,presence
0,SSP272,Archeops,4,0.70,Colorless,Pokémon,247.0,Lugia Archeops,5101,Alexander Flatos,...,United States,2023,10,14,BST-OBF,standard_2023,2023,4,14,1
1,SIT138,Lugia V,3,6.20,Colorless,Pokémon,247.0,Lugia Archeops,5101,Alexander Flatos,...,United States,2023,10,14,BST-OBF,standard_2023,2023,4,14,1
2,SIT139,Lugia VSTAR,3,8.71,Colorless,Pokémon,247.0,Lugia Archeops,5101,Alexander Flatos,...,United States,2023,10,14,BST-OBF,standard_2023,2023,4,14,1
3,LOR143,Snorlax,3,0.93,Colorless,Pokémon,247.0,Lugia Archeops,5101,Alexander Flatos,...,United States,2023,10,14,BST-OBF,standard_2023,2023,4,14,1
4,SSP250,Lumineon V,2,1.90,Water,Pokémon,247.0,Lugia Archeops,5101,Alexander Flatos,...,United States,2023,10,14,BST-OBF,standard_2023,2023,4,14,1


In [104]:
df.columns

Index(['id_card', 'name_card', 'amount_card', 'price_card', 'energy_type_card',
       'type_card', 'combo_type_id', 'combo_type_name', 'id_player',
       'name_player', 'country_player', 'all_time_score',
       'ranking_player_tournament', 'id_tournament', 'category_tournament',
       'name_tournament', 'region_tournament', 'country_tournament',
       'year_tournament', 'month_tournament', 'day_tournament',
       'valid_rotation_at_tournament', 'rotation_name', 'year_begin',
       'month_begin', 'day_begin', 'presence'],
      dtype='object')

In [64]:
df['combo_type_name'].value_counts()

combo_type_name
Lugia Archeops         17646
Lost Zone Box          10285
Gardevoir               7079
Mew Genesect            7030
Fusion Mew              4654
                       ...  
Lunatone Solrock          18
Ho-Oh                     17
Cramorant Porygon-Z       17
Hoopa Umbreon             16
Chandelure                14
Name: count, Length: 376, dtype: int64

In [105]:
df['category_tournament'].value_counts()

category_tournament
regional         52978
international    26623
others           20241
worlds            9218
national          5231
Name: count, dtype: int64

In [49]:

# Ensure the columns 'combo_type_name' and 'region' are present in the dataset
# df = df[['combo_type_name', 'region_tournament', 'energy_type_card']]

In [76]:
len(df)

114291

In [106]:
df = df.dropna(subset=['combo_type_name', 'region_tournament', 'category_tournament'])

In [107]:
len(df)

58365

In [85]:
# If 'presence' column doesn't exist, create it for the BinaryTarget
df['presence'] = (df['combo_type_name'] == 'Lugia Archeops').astype(int)

In [124]:
# Define the target for subgroup discovery
target = ps.BinaryTarget('presence', 1)

# Define the search space
combo_selectors = [ps.EqualitySelector('combo_type_name', value) for value in df['combo_type_name'].unique()]
region_selectors = [ps.EqualitySelector('region_tournament', value) for value in df['region_tournament'].unique()]
energy_selectors = [ps.EqualitySelector('category_tournament', value) for value in df['category_tournament'].unique() ]
search_space = combo_selectors + region_selectors + energy_selectors


In [121]:
search_space

[combo_type_name=='Charizard',
 combo_type_name=='Giratina LZ Box',
 combo_type_name=='Charizard Arceus',
 combo_type_name=='Arceus Umbreon',
 combo_type_name=='Gardevoir',
 combo_type_name=='Inteleon Urshifu',
 combo_type_name=='Miraidon Flaaffy',
 combo_type_name=='Lugia Archeops',
 combo_type_name=='Mew Genesect',
 combo_type_name=='Lost Zone Box',
 combo_type_name=='Arceus Duraludon Umbreon',
 combo_type_name=='Arceus',
 combo_type_name=='Charizard Gallade',
 combo_type_name=='Charizard LZ Box',
 combo_type_name=='Gardevoir Mewtwo',
 combo_type_name=='Arceus Giratina',
 combo_type_name=='Fusion Mew',
 combo_type_name=='Lost Box Charizard',
 combo_type_name=='Miraidon Regieleki',
 combo_type_name=='Goodra LZ Box',
 combo_type_name=='Blaziken Urshifu',
 combo_type_name=='Arceus Duraludon',
 combo_type_name=='Snorlax Stall',
 combo_type_name=='Kyurem',
 combo_type_name=='LZ Rayquaza Box',
 combo_type_name=='Vikavolt',
 combo_type_name=='Regis',
 combo_type_name=='Arceus Flying Pikachu

In [122]:

task = ps.SubgroupDiscoveryTask(df, target, search_space, result_set_size=15, depth=5, qf=ps.WRAccQF())

# Run the SD-Map algorithm
result = ps.SimpleDFS().execute(task)


In [123]:
# Display the results
for row in result.to_dataframe().itertuples():
    print(row)

Pandas(Index=0, quality=0.1166214262553025, subgroup=(combo_type_name=='Lugia Archeops'), size_sg=7867, size_dataset=58365, positives_sg=7867, positives_dataset=7867, size_complement=50498, relative_size_sg=0.13478968559924612, relative_size_complement=0.8652103144007539, coverage_sg=1.0, coverage_complement=0.0, target_share_sg=1.0, target_share_complement=0.0, target_share_dataset=0.13478968559924612, lift=7.4189652980805905)
Pandas(Index=1, quality=0.0, subgroup=True, size_sg=58365, size_dataset=58365, positives_sg=7867, positives_dataset=7867, size_complement=0, relative_size_sg=1.0, relative_size_complement=0.0, coverage_sg=1.0, coverage_complement=0.0, target_share_sg=0.13478968559924612, target_share_complement=nan, target_share_dataset=0.13478968559924612, lift=1.0)
